ComplianceGPT Answerer V0

**Model**: Qwen2.5-7B-Instruct-AWQ

**Data**: Canonical Clause Store (CCS) JSONL OF THREE COMPLIANCE FRAMEWORK (NIST rev4/5, HIPAA 2023/2024, PCI 4.0/4.0.1 + deltas + crosswalk).

**Logic**:

1.   loads/normalizes CCS → chunksList item
2.   loads/normalizes CCS → chunks
3.   retrieves clauses for a question
4. instructs model to output JSON-only, sentence-level citations
5. post-validates (enforces sentence citations, version pinning, ODP handling)


**Status codes**: OK | PARAMS_REQUIRED | NO_EVIDENCE.

1. Use transformers with AWQ weights.
Add dense retrieval.

In [ ]:
%pip install -qU autoawq accelerate transformers rank_bm25 tiktoken orjson
%pip install -qU sentence-transformers faiss-cpu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.3/74.3 kB 2.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.1/40.1 kB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.6/11.6 MB 43.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 486.6/486.6 kB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.4/31.4 MB 20.6 MB/s eta 0:00:00


In [ ]:
%pip install -qU json-repair

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Use a pipeline as a high-level helper
from transformers import pipeline

pipe = pipeline("text-generation", model="Qwen/Qwen2.5-7B-Instruct-AWQ")
messages = [
    {"role": "user", "content": "Who are you?"},
]
pipe(messages)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/841 [00:00<?, ?B/s]

You have loaded an AWQ model on CPU and have a CUDA/XPU device available, make sure to set your model on a GPU device in order to run your model.
We suggest you to set `dtype=torch.float16` for better efficiency on CUDA/XPU with AWQ.


model.safetensors.index.json: 0.00B [00:00, ?B/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

...

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

Device set to use cuda:0
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)


[{'generated_text': [{'role': 'user', 'content': 'Who are you?'},
   {'role': 'assistant',
    'content': 'I am Qwen, an AI assistant created by Alibaba Cloud. I am here to help with a variety of tasks and provide information on a wide range of topics. How can I assist you today?'}]}]

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch, random, numpy as np

MODEL_ID = "Qwen/Qwen2.5-7B-Instruct-AWQ"

def set_seeds(seed=42):
    random.seed(seed); np.random.seed(seed)
    try: torch.manual_seed(seed); torch.cuda.manual_seed_all(seed)
    except: pass
set_seeds(42)

tokenizer = AutoTokenizer.from_pretrained(MODEL_ID, use_fast=True)
model = AutoModelForCausalLM.from_pretrained(
    MODEL_ID,
    device_map="auto",
    torch_dtype=torch.float16
)

GEN_KW = dict(temperature=0.0, top_p=1.0, do_sample=False, max_new_tokens=400)


`torch_dtype` is deprecated! Use `dtype` instead!


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

System prompt:

You MUST NOT invent values for tokens like {{ insert: param, … }}. If a value is not provided by the Org Profile, keep the token verbatim in quoted citations and add an “ASK” list of missing parameters using the provided friendly_label and ask_prompt. Never guess numbers or durations.

4) Load JSONL (latest versions by default) + chunk

In [ ]:
import os, re, json, glob, itertools
from typing import List, Dict, Any
import tiktoken

DATA_ROOT = "/content/drive/MyDrive/compliance_data/ccs"
PREFERRED = {"NIST SP 800-53": "rev5", "HIPAA": "2024", "PCI-DSS": "4_0"}
CHUNK_TOKS, CHUNK_OVERLAP = 800, 120
enc = tiktoken.get_encoding("cl100k_base")

def discover_latest_paths():
    paths = []
    paths += glob.glob(f"{DATA_ROOT}/hipaa/*2024*_ccs.jsonl")
    paths += glob.glob(f"{DATA_ROOT}/nist800-53/rev5.jsonl")
    paths += glob.glob(f"{DATA_ROOT}/pci_dss/pcidss_v4_0.jsonl")
    return sorted(paths)

def load_rows(paths:List[str])->List[Dict[str,Any]]:
    rows=[]
    for p in paths:
        with open(p,"r",encoding="utf-8") as f:
            for line in f:
                row = json.loads(line)
                row["framework"]=row.get("framework","")
                row["version"]=row.get("version","")
                row["control_id"]=row.get("control_id","")
                row["title"]=row.get("title","")
                row["text"]=row.get("text","")
                row["doc_id"]=f"{row['framework']}/{row['version']}/{row['control_id']}"
                rows.append(row)
    return rows

def chunkify(row, max_toks=CHUNK_TOKS, overlap=CHUNK_OVERLAP):
    base = f"{row['title']} — {row['text']}".strip()
    toks = enc.encode(base)
    if len(toks) <= max_toks:
        return [row | {"chunk_ix":0, "chunk_text":base}]
    out=[]; i=0
    while i < len(toks):
        seg = toks[i:i+max_toks]
        out.append(row | {"chunk_ix":len(out), "chunk_text":enc.decode(seg)})
        i += max_toks - overlap
    return out

paths = discover_latest_paths()
rows = load_rows(paths)
chunks = list(itertools.chain.from_iterable(chunkify(r) for r in rows))
len(paths), len(rows), len(chunks)


(3, 1626, 1626)

5) BM25 index (+ ODP detector)

In [ ]:
from rank_bm25 import BM25Okapi
import numpy as np

def detect_odp(text:str)->bool:
    t = text.lower()
    triggers = [
        "organization-defined","organization defined","odp",
        "{organization-defined}","org-defined parameter","parameter defined by the organization"
    ]
    return any(x in t for x in triggers)

corpus_lc = [re.sub(r"\s+"," ", c["chunk_text"]).lower() for c in chunks]
bm25 = BM25Okapi([c.split() for c in corpus_lc])

def retrieve(query:str, topk=8, preferred=PREFERRED):
    tokens = query.lower().split()
    scores = bm25.get_scores(tokens)
    order = np.argsort(scores)[::-1]
    hits=[]
    for i in order:
        c = chunks[i]
        want = preferred.get(c["framework"])
        if want and c["version"] != want:
            continue
        hits.append({
            "idx": int(i),
            "score": float(scores[i]),
            "framework": c["framework"],
            "version": c["version"],
            "clause_id": c["control_id"],
            "title": c["title"],
            "chunk_text": c["chunk_text"],
            "doc_id": c["doc_id"],
            "has_odp": detect_odp(c["chunk_text"])
        })
        if len(hits) >= topk: break
    return hits




6) Build “citation-contract” prompt

In [ ]:
import re, json

SYSTEM = """You are ComplianceGPT.
Answer ONLY from the provided context.
Return STRICT JSON with keys: answer (array of {sentence, citations}), citations (array of {framework, clause_id, version, spans}), odps_encountered (array), ask_list (array), status, audit.
Every sentence MUST include at least one citation like "NIST SP 800-53 AC-2(3)@rev5".
If an Organization-Defined Parameter (ODP) is needed and not present, set status="PARAMS_REQUIRED" and add an ask_list item.
If there is no support, set status="NO_EVIDENCE" and do not invent content.
Output JSON ONLY. No prose, no markdown fences, no extra text.
You MUST NOT invent values for tokens like {{ insert: param, … }}.
If a value is not provided by the Org Profile, keep the token verbatim in quoted citations and add an “ASK” list of missing parameters using the provided friendly_label and ask_prompt.
Never guess numbers or durations.
"""

def build_context(hits):
    lines=[]
    for h in hits:
        tag = f"{h['framework']} {h['clause_id']}@{h['version']}"
        txt = h["chunk_text"][:1200].replace("\n"," ")
        lines.append(f"[{tag}] {h['title']} — {txt}")
    return "\n\n".join(lines)

def build_user_prompt(question:str, org_profile:dict, hits:list)->str:
    ctx = build_context(hits)
    return f"""QUESTION:
{question}

ORG_PROFILE (JSON):
{json.dumps(org_profile or {}, ensure_ascii=False)}

CONTEXT (retrieved clauses):
{ctx}

RESPONSE FORMAT (STRICT JSON):
{{
  "answer":[{{"sentence":"...", "citations":["Framework CLAUSE@VERSION"]}}],
  "citations":[{{"framework":"...", "clause_id":"...", "version":"...", "spans":[{{"doc_id":"...", "start":0, "end":100}}]}}]],
  "odps_encountered":[{{"framework":"...","clause_id":"...","parameter":"..."}}]],
  "ask_list":[{{"framework":"...","clause_id":"...","parameter":"...","prompt":"..."}}]],
  "status":"OK|PARAMS_REQUIRED|NO_EVIDENCE",
  "audit": {{"note":"no legal disclaimers"}}
}}"""

def _strip_code_fences(s:str)->str:
    s = s.strip()
    s = re.sub(r"^```(?:json)?\s*", "", s, flags=re.I)
    s = re.sub(r"\s*```$", "", s)
    return s.strip()

def safe_json_loads_maybe_slice(s:str):
    s = _strip_code_fences(s)
    start = s.find("{")
    end   = s.rfind("}")
    if start != -1 and end != -1 and end > start:
        s = s[start:end+1]
    try:
        return json.loads(s)
    except:
        try:
            import orjson
            return orjson.loads(s)
        except:
            return None

def build_messages(question: str, org_profile: dict, hits: list):
    return [
        {"role": "system", "content": SYSTEM},
        {"role": "user",   "content": build_user_prompt(question, org_profile, hits)},
    ]




6.1 add a json parser helpfer

In [ ]:
import re, json
from json_repair import repair_json

def _strip_code_fences(s: str) -> str:
    s = s.strip()
    s = re.sub(r"^```(?:json)?\s*", "", s, flags=re.I)
    s = re.sub(r"\s*```$", "", s)
    return s.strip()

def _slice_outer_braces(s: str) -> str:
    # Find the *longest* plausible JSON object by matching braces
    s = _strip_code_fences(s)
    # Quick path: use first '{' to last '}' as a window
    if "{" in s and "}" in s:
        start = s.find("{")
        end   = s.rfind("}")
        if end > start:
            return s[start:end+1]
    return s

def robust_parse_json(text: str):
    # 1) direct parse
    try:
        return json.loads(text)
    except Exception:
        pass
    # 2) slice by outer braces + parse
    sliced = _slice_outer_braces(text)
    try:
        return json.loads(sliced)
    except Exception:
        pass
    # 3) repair then parse
    try:
        repaired = repair_json(sliced)
        return json.loads(repaired)
    except Exception:
        # final fallback: return None so caller can handle NO_EVIDENCE
        return None


7) Generate JSON and post-validate (enforce contract)

In [ ]:
import re, json
from typing import List, Dict, Any

# Optional accelerators for parsing; safely imported if available.
try:
    import orjson as _orjson
except Exception:
    _orjson = None

try:
    from json_repair import repair_json as _repair_json
except Exception:
    _repair_json = None


# --- Robust JSON parsing helpers ---

def _strip_code_fences(s: str) -> str:
    # Removes ```json ... ``` or ``` ... ``` fences if the model added them
    s = s.strip()
    s = re.sub(r"^```(?:json)?\s*", "", s, flags=re.I)
    s = re.sub(r"\s*```$", "", s)
    return s.strip()

def _slice_outer_braces(s: str) -> str:
    # Takes the longest plausible JSON object substring from first '{' to last '}'
    s = _strip_code_fences(s)
    if "{" in s and "}" in s:
        start = s.find("{")
        end = s.rfind("}")
        if end > start:
            return s[start:end+1]
    return s

def robust_parse_json(text: str):
    # 1) direct parse
    try:
        return json.loads(text)
    except Exception:
        pass
    # 2) slice by outer braces, then parse
    sliced = _slice_outer_braces(text)
    try:
        return json.loads(sliced)
    except Exception:
        pass
    # 3) fast parser if available
    if _orjson is not None:
        try:
            return _orjson.loads(sliced)
        except Exception:
            pass
    # 4) attempt repair (if library present), then parse
    if _repair_json is not None:
        try:
            repaired = _repair_json(sliced)
            return json.loads(repaired)
        except Exception:
            pass
    # 5) give up and return None; caller decides on NO_EVIDENCE
    return None


# --- Tokenization for chat turns (builds attention_mask) ---

def _tokenize_chat(chat_turns: List[Dict[str, str]], add_generation_prompt: bool):
    """
    chat_turns example:
      [{"role":"system","content": ...},
       {"role":"user","content": ...}]
    Returns a dict with input_ids and attention_mask on the model device.
    """
    # Render the chat to a single text prompt using the model's template
    prompt_text = tokenizer.apply_chat_template(
        chat_turns,
        add_generation_prompt=add_generation_prompt,
        tokenize=False  # produce plain text; tokenize in the next step to get attention_mask
    )
    # Tokenize with padding to obtain a reliable attention_mask
    model_inputs = tokenizer(
        prompt_text,
        return_tensors="pt",
        padding=True,
        return_attention_mask=True,
    )
    # Move to model device
    return {k: v.to(model.device) for k, v in model_inputs.items()}


# --- Generation with fallbacks to enforce JSON-only output ---

def generate_contract_json(messages: List[Dict[str, str]]) -> Dict[str, Any] | None:
    """
    Generates JSON using two strategies with robust parsing:
      S1: normal chat generation (add_generation_prompt=True)
      S2: assistant '{' seed to force JSON continuation (add_generation_prompt=False)
    Returns a parsed dict on success, or None on failure.
    Assumes globals: tokenizer, model, GEN_KW are defined.
    """
    def _run(model_inputs: Dict[str, Any], gen_kw: Dict[str, Any]) -> str:
        with torch.no_grad():
            out = model.generate(**model_inputs, **gen_kw)
        completion = tokenizer.decode(
            out[0][model_inputs["input_ids"].shape[-1]:],
            skip_special_tokens=True
        )
        return completion

    # Shared generation kwargs (deterministic). Remove top_k if present to silence warnings.
    gen_kw = GEN_KW.copy()
    gen_kw.pop("top_k", None)

    # Prefer stopping at the assistant end token if available.
    im_end_id = tokenizer.convert_tokens_to_ids("<|im_end|>")
    if im_end_id is not None:
        gen_kw["eos_token_id"] = im_end_id

    # ---- Strategy 1: normal chat
    s1_inputs = _tokenize_chat(messages, add_generation_prompt=True)
    s1_txt = _run(s1_inputs, gen_kw)
    s1 = robust_parse_json(s1_txt)
    if isinstance(s1, dict):
        return s1

    # ---- Strategy 2: assistant '{' seed to force JSON object continuation
    msgs_seed = messages + [{"role": "assistant", "content": "{"}]
    s2_inputs = _tokenize_chat(msgs_seed, add_generation_prompt=False)
    s2_txt = _run(s2_inputs, gen_kw)
    if not s2_txt.strip().startswith("{"):
        s2_txt = "{" + s2_txt
    s2 = robust_parse_json(s2_txt)
    if isinstance(s2, dict):
        return s2

    # Nothing parsed cleanly
    return None


8. Function

In [ ]:
def answer_citation_contract(question:str, org_profile:dict=None, preferred_versions:dict=None, topk:int=8)->dict:
    org_profile = org_profile or {}
    preferred_versions = preferred_versions or PREFERRED
    hits = retrieve(question, topk=topk, preferred=preferred_versions)
    if not hits:
        return {"answer":[],"citations":[],"odps_encountered":[],"ask_list":[],"status":"NO_EVIDENCE","audit":{"reason":"no retrieval"}}
    prompt = build_user_prompt(question, org_profile, hits)
    msgs = build_messages(question, org_profile, hits)
    raw = generate_contract_json(msgs)
    if not isinstance(raw, dict):
        return {"answer":[],"citations":[],"odps_encountered":[],"ask_list":[],"status":"NO_EVIDENCE","audit":{"reason":"model returned non-JSON"}}
    return enforce_contract(raw, hits, org_profile)


9. Smoke Test

In [ ]:
tests = [
    ("How should inactive accounts be handled?", {}),
    ("What does HIPAA require for risk analysis?", {}),
    ("Compare log retention requirements for NIST and PCI.", {})
]
for q, prof in tests:
    out = answer_citation_contract(q, org_profile=prof)
    print(q)
    print(json.dumps(out, indent=2)[:2000])
    print("status=", out.get("status"), "\n---\n")

The following generation flags are not valid and may be ignored: ['temperature', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


How should inactive accounts be handled?
{
  "answer": [],
  "citations": [],
  "odps_encountered": [],
  "ask_list": [],
  "status": "NO_EVIDENCE",
  "audit": {
    "reason": "citations not in retrieved pool"
  }
}
status= NO_EVIDENCE 
---

What does HIPAA require for risk analysis?
{
  "answer": [],
  "citations": [],
  "odps_encountered": [],
  "ask_list": [],
  "status": "NO_EVIDENCE",
  "audit": {
    "reason": "no citations on a sentence"
  }
}
status= NO_EVIDENCE 
---

Compare log retention requirements for NIST and PCI.
{
  "answer": [],
  "citations": [],
  "odps_encountered": [],
  "ask_list": [],
  "status": "NO_EVIDENCE",
  "audit": {
    "reason": "citations not in retrieved pool"
  }
}
status= NO_EVIDENCE 
---

